Part Seven:

---
Transfer Learning
*   Load Spectrogram Images & Encode the Labels
*   Split Data into Training, Testing & Validation Data
*   Import VGG16 Model & Modify Accordingly
*   Classify with Modified VGG16 Model




**Import Headers**

In [0]:
import os
import pickle as pkl
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%tensorflow_version 1.x
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from sklearn import metrics
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import callbacks
from tensorflow.contrib.keras import regularizers
from sklearn.utils.class_weight import compute_class_weight

**Load Spectrogram Images & List Genres**

In [0]:
with open('/content/drive/My Drive/Project/Data/Working_Data/Image.pickle','rb') as f:
    x = pkl.load(f)
with open('/content/drive/My Drive/Project/Data/Working_Data/Genre.pickle','rb') as f:
    y = pkl.load(f)
genres = os.listdir("/content/drive/My Drive/Project/Data/Working_Data/Spectogram_Images")
print("The list of genres are:",genres)
le = preprocessing.LabelEncoder()
y_label_encoded = le.fit_transform(y)
cl_weight = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_label_encoded), y = y_label_encoded)

**Split Data into Train, Test & Validation Data**

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y_label_encoded, test_size=0.2, random_state=20)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=20)
print("The shape of x-train is:",X_train.shape)
print("The shape of y-train is:",Y_train.shape)
print("The shape of x-test is:",X_test.shape)
print("The shape of y-test is:",Y_test.shape)
print("The shape of x-validation is:",X_val.shape)
print("The shape of y_validation is:",Y_val.shape)

**Import VGG16 Model**

In [0]:
conv_base = tf.contrib.keras.applications.VGG16(include_top = False, weights = 'imagenet', input_shape = (288, 432, 3))
print("Original VGG16 Model Summary:")
conv_base.summary()

**Modify VGG16 Model**

In [0]:
NUM_CLASSES = 10
BATCH_SIZE = 32
L2_LAMBDA = 0.001
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten()) #Flatten output and send it to MLP
#1-layer MLP with Dropout
model.add(layers.Dense(512, name='dense_1', kernel_regularizer=regularizers.l2(L2_LAMBDA)))
model.add(layers.Dropout(rate=0.3, name='dropout_1')) #Can try varying dropout rates
model.add(layers.Activation(activation='relu', name='activation_1'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax', name='dense_output'))
conv_base.trainable = False
print("Modified VGG16 Model Summary:")
model.summary()

**Classify with Modified VGG16 Model**

*Function to Generate Batches*

In [0]:
def batch_generator(X,Y, BATCH_SIZE):
    L = len(Y)
    while True:
        batch_start = 0
        batch_end = BATCH_SIZE
        while batch_start < L:
            limit = min(batch_end, L)
            X_file_list = X[batch_start : limit]
            Y_file_list = Y[batch_start : limit]
            batch_img_array, batch_label_array = (X_file_list,Y_file_list)
            yield (batch_img_array, batch_label_array)     
            batch_start += BATCH_SIZE   
            batch_end += BATCH_SIZE

*Function to Draw Confusion Matrix*

In [0]:
def draw_confusion_matrix(y_test,y_pred,epochs):
  cm = metrics.confusion_matrix(y_test, y_pred)
  plt.figure(figsize=(9,9))
  sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues');
  plt.ylabel('Actual label');
  plt.xlabel('Predicted label');
  title = 'Transfer Learning with VGG16 Model for '+str(epochs)+' epochs'
  plt.title(title, size = 15);
  plt.show()

*Compile & Train the Model for 10 epochs*

In [0]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
STEPS_PER_EPOCH = len(Y_train)//BATCH_SIZE
VAL_STEPS = len(Y_val)//BATCH_SIZE
history = model.fit_generator(generator = batch_generator(X_train, Y_train, BATCH_SIZE), epochs = 10, steps_per_epoch = STEPS_PER_EPOCH, class_weight = cl_weight,
                              validation_data = batch_generator(X_val, Y_val, BATCH_SIZE), validation_steps = VAL_STEPS)

*Plot the Training & Validation Curve*

In [0]:
d = history.history
training_loss = d['loss']
validation_loss = d['val_loss']
plt.plot(training_loss, label = 'Train Loss')
plt.plot(validation_loss, label = 'Validation Loss')
plt.legend()
plt.show()

*Calculate Accuracy of the Model & Print the Confusion Matrix*

In [0]:
Y_pred = model.predict(X_test)
y_test = Y_test
y_pred = []
c=0
for i in range(len(Y_pred)):
    y_pred.append(np.argmax(Y_pred[i]))
    if(np.argmax(Y_pred[i]) == Y_test[i]):
        c+=1
accuracy = c/len(Y_pred) * 100
print("The accuracy is:",str(accuracy),"%.")
draw_confusion_matrix(y_test, y_pred,10)

*Compile & Train the Model for 100 epochs*

In [0]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
STEPS_PER_EPOCH = len(Y_train)//BATCH_SIZE
VAL_STEPS = len(Y_val)//BATCH_SIZE
history = model.fit_generator(generator = batch_generator(X_train, Y_train, BATCH_SIZE), epochs = 100, steps_per_epoch = STEPS_PER_EPOCH, class_weight = cl_weight,
                              validation_data = batch_generator(X_val, Y_val, BATCH_SIZE), validation_steps = VAL_STEPS)

*Plot the Training & Validation Curve*

In [0]:
d = history.history
training_loss = d['loss']
validation_loss = d['val_loss']
plt.plot(training_loss, label = 'Train Loss')
plt.plot(validation_loss, label = 'Validation Loss')
plt.legend()
plt.show()

*Calculate Accuracy of the Model & Print the Confusion Matrix*

In [0]:
Y_pred = model.predict(X_test)
y_test = Y_test
y_pred = []
c=0
for i in range(len(Y_pred)):
    y_pred.append(np.argmax(Y_pred[i]))
    if(np.argmax(Y_pred[i]) == Y_test[i]):
        c+=1
accuracy = c/len(Y_pred) * 100
print("The accuracy is:",str(accuracy),"%.")
draw_confusion_matrix(y_test, y_pred,100)
print("Part Seven Successful!")